# Unit tests

Unit tests are one of the most important inddicators of professional code. They are a way to use your code with various inputs, common and corner cases, in an automated, programmatic way. The tool of our choice will by `pytest`

In [ ]:
!pip install pytest

### Create a sample library

Let's create a "library" to test

In [ ]:
%%writefile name_reverser.py
# This library parses names and presents them in a professional, reverse name order

def name_reverse_order(full_name):
    first, last = full_name.split(' ')
    return f'{last}, {first}'


In [ ]:
# %load name_reverser.py
# This library parses names and presents them in a professional, reverse name order

def name_reverse_order(full_name):
    first, last = full_name.split(' ')
    return f'{last}, {first}'


Let's do a basic sanity check. Does it work?

In [ ]:
name_reverse_order("Michael Jordan")

Great! Let's try another

In [ ]:
name_reverse_order("John Fitzgerald Kennedy")

Yikes! Looks like our logic doesn't work.

As we are coding, our testing should not just be arbitrary, done in a manual way. We can create another file which will load our code, exercise it and return the results.

We can be more systematic about this by creating a _unit test_ and run it via the `pytest` command

### Let's create some tests

In [ ]:
%%writefile tests/test_name_reverser.py
import pytest
from name_reverser import name_reverse_order

def test_name_reverse_order_normal():
    rslt = name_reverse_order("Michael Jordan")
    assert rslt == "Jordan, Michael"
    

In [ ]:
!pytest tests/test_name_reverser.py

#### Multiple tests in a file
Let's add more tests

In [ ]:
%%writefile tests/test_name_reverser.py
import pytest
from name_reverser import name_reverse_order

def test_name_reverse_order_normal():
    rslt = name_reverse_order("Michael Jordan")
    assert rslt == "Jordan, Michael"

    rslt = name_reverse_order("Lebron James")
    assert rslt == "James, Lebron"

def test_name_reverse_order_middle_names():
    rslt = name_reverse_order("John F Kennedy")
    assert rslt == "Kennedy, John F"

    rslt = name_reverse_order("Jean Luc Picard")
    assert rslt == "Picard, Jean Luc"


In [ ]:
!pytest tests/test_name_reverser.py

#### Corner cases that _should_ produce errors

In fact, we should add silly corner cases.

In [ ]:
%%writefile tests/test_name_reverser.py
import pytest
from name_reverser import name_reverse_order

def test_name_reverse_order_normal():
    rslt = name_reverse_order("Michael Jordan")
    assert rslt == "Jordan, Michael"

    rslt = name_reverse_order("Lebron James")
    assert rslt == "James, Lebron"

def test_name_reverse_order_bad_inputs():

    # Empty string
    with pytest.raises(ValueError):
        rslt = name_reverse_order("")
    

In [ ]:
!pytest tests/test_name_reverser.py

The error producing test passed because we _expect_ and _want_ it to produce an error in some cases

#### When expected errors aren't found
Let's see what happens when an expected error is not found

In [ ]:
%%writefile tests/test_name_reverser.py
import pytest
from name_reverser import name_reverse_order

def test_name_reverse_order_normal():
    rslt = name_reverse_order("Michael Jordan")
    assert rslt == "Jordan, Michael"

    rslt = name_reverse_order("Lebron James")
    assert rslt == "James, Lebron"

def test_name_reverse_order_bad_inputs():

    # Empty string
    with pytest.raises(ValueError):
        rslt = name_reverse_order("")
        
    # Too normal?
    with pytest.raises(ValueError):
        rslt = name_reverse_order("George Washington")
    

In [ ]:
!pytest tests/test_name_reverser.py

#### Multiple test files

In [ ]:
%%writefile tests/test_name_reverser.py
import pytest
from name_reverser import name_reverse_order

def test_name_reverse_order_normal():
    rslt = name_reverse_order("Michael Jordan")
    assert rslt == "Jordan, Michael"

    rslt = name_reverse_order("Lebron James")
    assert rslt == "James, Lebron"

def test_name_reverse_order_bad_inputs():

    # Empty string
    with pytest.raises(ValueError):
        rslt = name_reverse_order("")
        

In [ ]:
%%writefile tests/test_name_reverser_part_deux.py
import pytest
from name_reverser import name_reverse_order

def test_name_reverse_order_normal():
    rslt = name_reverse_order("Michael Jordan")
    assert rslt == "Jordan, Michael"

    rslt = name_reverse_order("Lebron James")
    assert rslt == "James, Lebron"

def test_name_reverse_order_bad_inputs():

    # Empty string
    with pytest.raises(ValueError):
        rslt = name_reverse_order("")
        

In [ ]:
!pytest tests/

### Tests before code: test driven development

A very interesting technique is to write unit test **before** writing the actual code! This turns unit tests from a QA related task to a spec, which the final program must pass.

Senior developers are write unit tests and hand them to more junion developers. Junior developers now know _exactly_ what the interface will look like, what the inputs will look like and what the scope of the project will be.

This is also a fantastic technique to get the programmers to think about how their code will be used.

### Code coverage tools track which parts of code are NOT covered by tests

In [ ]:
!pip install coverage

In [ ]:
%%writefile name_reverser.py
# This library parses names and presents them in a professional, reverse name order

def name_reverse_order(full_name):
    if full_name == "": # Handle the case where an empty string is passed
        return ""
    else:
        first, last = full_name.split(' ')
        return f'{last}, {first}'


In [ ]:
%%writefile tests/test_name_reverser.py
import pytest
from name_reverser import name_reverse_order

def test_name_reverse_order_normal():
    rslt = name_reverse_order("Michael Jordan")
    assert rslt == "Jordan, Michael"

    rslt = name_reverse_order("Lebron James")
    assert rslt == "James, Lebron"



In [ ]:
!pytest tests/test_name_reverser.py

In [ ]:
!coverage run -m pytest tests/test_name_reverser.py

In [ ]:
!coverage report

#### A visual report!

In [ ]:
!coverage html

Now load the file htmlcov\index.html from your _browser_ to see the source code, annotated with which lines were not covered by your tests!

### Make your test run after every commit on GitHub

```yaml
name: Run Unit Test via Pytest  
  
on: [push]  
  
jobs:  
  build:  
    runs-on: ubuntu-latest  
    strategy:  
      matrix:  
        python-version: ["3.10", "3.11", "3.12"]  
  
    steps:  
      - uses: actions/checkout@v3  
      - name: Set up Python ${{ matrix.python-version }}  
        uses: actions/setup-python@v4  
        with:  
          python-version: ${{ matrix.python-version }}  
      - name: Install dependencies  
        run: |  
          python -m pip install --upgrade pip  
          if [ -f requirements.txt ]; then pip install -r requirements.txt; fi  
      - name: Test with pytest  
        run: |  
          pytest tests/ 
      - name: Generate Coverage Report  
        run: |  
          coverage report -m

```

source: https://pytest-with-eric.com/integrations/pytest-github-actions/

Keep this file in `.github/workflows/run_unit_tests.yml`